## Let's use K-fold 

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io

In [6]:
train = scipy.io.loadmat('train_32x32.mat')
test  = scipy.io.loadmat('test_32x32.mat')

In [7]:
train['X'] = train['X'][np.newaxis,...]
train_data = np.swapaxes(train['X'],0,4).squeeze()

test['X'] = test['X'][np.newaxis,...]
test_data = np.swapaxes(test['X'],0,4).squeeze()

In [14]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

train_data /= 255.0
test_data /= 255.0

#### Convert rgb to grayscale

In [15]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [16]:
X_train = np.zeros((73257, 32, 32))
for i in range(train_data.shape[0]):
    X_train[i] = rgb2gray(train_data[i])

In [17]:
X_test = np.zeros((26032, 32, 32))
for i in range(test_data.shape[0]):
    X_test[i] = rgb2gray(test_data[i])

In [18]:
X_train = X_train.reshape(X_train.shape[0],32, 32, 1)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 1)

-----------------------

In [19]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import BatchNormalization

Using TensorFlow backend.


-----------------------------

In [20]:
y_train = keras.utils.to_categorical(train['y'][:,0])[:,1:]
y_test = keras.utils.to_categorical(test['y'][:,0])[:,1:]

-----------------------------

In [21]:
from sklearn.model_selection import StratifiedKFold

In [55]:
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=111)
cvscores = []

In [56]:
from keras.optimizers import Adam
adam = Adam(decay=1e-6)

In [57]:
y_ = train['y'].reshape(-1)

In [58]:
for train_indices, val_indices in kfold.split(X_train,y_):
    
    model5 = Sequential()
    model5.add(Conv2D(32, (3,3), strides=(1,1), padding='same', input_shape=(32,32,1)))
    model5.add(Activation('relu'))
    model5.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model5.add(Dropout(0.35))
    model5.add(Conv2D(64, (3,3), strides=(1,1),padding='same'))
    model5.add(Activation('relu'))
    model5.add(Conv2D(64, (3,3), strides=(1,1),padding='same'))
    model5.add(Activation('relu'))
    model5.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
    model5.add(Dropout(0.35))
    model5.add(Flatten())
    model5.add(Dense(256))
    model5.add(Activation('relu'))
    model5.add(Dropout(0.35))
    model5.add(Dense(10))
    model5.add(Activation('softmax'))
    
    model5.compile(loss='categorical_crossentropy', optimizer=adam , metrics=['accuracy'])
    model5.fit(X_train[train_indices], y_train[train_indices], epochs=15, batch_size=128, verbose=1)
    scores = model5.evaluate(X_train[val_indices], y_train[val_indices], verbose=0)
    print("%s: %.2f%%" % (model5.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/15
54938/54938 [==============================] - 336s - loss: 1.6069 - acc: 0.4438   
Epoch 2/15
54938/54938 [==============================] - 306s - loss: 0.6970 - acc: 0.7852   
Epoch 3/15
54938/54938 [==============================] - 309s - loss: 0.5730 - acc: 0.8237   
Epoch 4/15
54938/54938 [==============================] - 307s - loss: 0.5120 - acc: 0.8432   
Epoch 5/15
54938/54938 [==============================] - 308s - loss: 0.4701 - acc: 0.8558   
Epoch 6/15
54938/54938 [==============================] - 306s - loss: 0.4370 - acc: 0.8665   
Epoch 7/15
54938/54938 [==============================] - 305s - loss: 0.4104 - acc: 0.8743   
Epoch 8/15
54938/54938 [==============================] - 310s - loss: 0.3931 - acc: 0.8792   
Epoch 9/15
54938/54938 [==============================] - 306s - loss: 0.3695 - acc: 0.8867   
Epoch 10/15
54938/54938 [==============================] - 306s - loss: 0.3542 - acc: 0.8920   
Epoch 11/15
54938/54938 [========================